In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import utilys as ut
import os
from scipy.misc import imread,imresize

In [2]:
width = 20

# gather data paths
subfolds = ut.extend_children('data','')
datafolds = [subfolds[0],subfolds[3]]
alphabets = ut.extend_generation(datafolds,'')
charpaths = ut.extend_generation(alphabets,'')
chars_dataset = [v.split('/')[2]+'/'+v.split('/')[3] for v in charpaths]

# index-value conversion dictionaries for character set
i2v = {i:v for i, v in enumerate(chars_dataset)}
v2i = {v:i for i, v in enumerate(chars_dataset)}

# get size of dataset
mc_dataset = len(charpaths)
print(mc_dataset,'total character classes')

# train split
mc_train = 1423
chars_train = chars_dataset[:mc_train]
classes_train = [v2i[v] for v in chars_train]

# validation split
mc_val = mc_dataset-mc_train
chars_val = chars_dataset[-mc_val:]
classes_val = [v2i[v] for v in chars_val]

print('%s characters assigned for training, %s characters assigned for validation'%(mc_train,mc_val))

1623 total character classes
1423 characters assigned for training, 200 characters assigned for validation


In [3]:
# load images from character paths
imgs_dataset = []
for i, charfold in enumerate(charpaths):
    if i%200==0:
        print('%s/%s character folders loaded'%(i,mc_dataset))
    imgs_dataset.append([ ut.load_image(imgpath,(width,width))/127.5-1 for imgpath in ut.extend_children(charfold,'.png') ] )
# access imgs_dataset [ character index ] [ sample index ] [ row,col ]

# split images between train and validation sets
imgs_train = imgs_dataset[:mc_train]
imgs_val = imgs_dataset[-mc_val:]

0/1623 character folders loaded
200/1623 character folders loaded
400/1623 character folders loaded
600/1623 character folders loaded
800/1623 character folders loaded
1000/1623 character folders loaded
1200/1623 character folders loaded
1400/1623 character folders loaded
1600/1623 character folders loaded


In [4]:
#classes_val, imgs_val = ut.shuffle_xy(classes_val,imgs_val,7)

n_classes = 5
learning_rate = 1e-4
batch_size = 16

n_inputs = width*width
n_hnodes = 200
latent_dim = 100
n_outputs = n_classes


weights = lambda shape,name: tf.Variable(tf.truncated_normal(shape, stddev=0.1),name=name)
biases = lambda shape,name: tf.Variable(tf.constant(0.1, shape=shape),name=name)
conv2d = lambda x, W: tf.nn.conv2d(x, W, strides=[1, 2, 2, 1], padding='SAME')
max_pool = lambda x: tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [5]:
import tensorflow as tf
from LSN import Latent_Similarity_Network

X = tf.placeholder(tf.float32, shape=[batch_size, None, width, width])
y = tf.placeholder(tf.float32, shape=[batch_size, None, n_classes])

X_fc = tf.reshape(X,[-1,width*width])
y_tr = tf.transpose(y,perm=[1,0,2])

W_d1 = weights([n_inputs,n_hnodes],'W_d1')
b_d1 = biases([n_hnodes],'b_d1')
W_d2 = weights([n_hnodes,latent_dim],'W_d2')
b_d2 = biases([latent_dim],'b_d2')

h_d1 = tf.nn.tanh(tf.matmul(X_fc,W_d1)+b_d1)
h_d2 = tf.nn.tanh(tf.matmul(h_d1,W_d2)+b_d2)
h_fc = tf.reshape(h_d2,[-1,7*7*4])

X_lsn = tf.transpose(tf.reshape(h_fc,[batch_size,-1,latent_dim]),perm=[1,0,2])

LSN0 = Latent_Similarity_Network(latent_dim,n_classes,batch_size,X_lsn,y_tr)

In [6]:
M, h = LSN0['M'], LSN0['h']

print(X_lsn.get_shape(),y_tr.get_shape(),h.get_shape())

xentropy = tf.nn.softmax_cross_entropy_with_logits(labels=y_tr,logits=h)
loss = tf.reduce_mean(xentropy)

trainable_variables = tf.trainable_variables()
optimizer = tf.train.AdamOptimizer(learning_rate)
grads, _ = tf.clip_by_global_norm(tf.gradients(loss,trainable_variables),30)
train_step = optimizer.apply_gradients(zip(grads,trainable_variables))

correct_prediction = tf.equal(tf.argmax(h, -1), tf.argmax(y_tr, -1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

tf.summary.scalar('loss',loss)
tf.summary.scalar('accuracy',accuracy)
tf.summary.scalar('memory_mean',tf.reduce_mean(M))
merged = tf.summary.merge_all()

(?, 16, 100) (?, 16, 5) (?, 16, 5)


In [7]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
savedir = './saver/LSN2'
saver = tf.train.Saver()
writer = tf.summary.FileWriter(savedir+'/tb')

In [8]:
import time
import os

n_epochs = 400

if False:
    saver.restore(sess, os.path.join(savedir,'400/Model.ckpt'))

with sess.as_default():
    for epoch in range(n_epochs):
        print('Epoch {} starting..'.format(epoch+1))
        epoch_start = time.time()
        classes_epoch, imgs_epoch = ut.shuffle_xy(classes_train,imgs_train)
        
        n_batches = len(classes_epoch)//(n_classes*batch_size)
        for batch in range(n_batches):
            classes_batch = classes_epoch[batch*n_classes*batch_size:(batch+1)*n_classes*batch_size]
            imgs_batch = imgs_epoch[batch*n_classes*batch_size:(batch+1)*n_classes*batch_size]
            
            Xl_batch, yl_batch = [], []
            for episode in range(batch_size):
                imgs_ep = imgs_batch[episode*n_classes:(episode+1)*n_classes]
                Xl_ep, yl_ep = [], []
                for ind, cat in enumerate(imgs_ep):
                    for arr in cat:
                        Xl_ep.append(arr)
                        yl_ep.append(ut.one_hot(ind,n_classes))
                Xl_shuff, yl_shuff = ut.shuffle_xy(Xl_ep,yl_ep)
                X_arr, y_arr = np.asarray(Xl_shuff), np.asarray(yl_shuff)
                Xl_batch.append(X_arr)
                yl_batch.append(y_arr)
            X_train, y_train = np.asarray(Xl_batch), np.asarray(yl_batch)
            
            _, loss_train, accuracy_train, summary = sess.run((train_step,loss,accuracy,merged),feed_dict={X:X_train,y:y_train})
            print('Epoch %d'%(epoch+1),'/ Batch (%d/%d)'%(batch+1,n_batches),'/ Accuracy %.4f / Loss %f'%(accuracy_train,loss_train))
            writer.add_summary(summary,(epoch*n_batches+batch)*batch_size)
        epoch_end = time.time()
        time_elapsed = epoch_end-epoch_start
        print('Epoch {} complete,'.format(epoch+1),time_elapsed,'seconds elapsed')
        if (epoch+1)%40 == 0:
            iterdir = os.path.join(savedir,str(epoch+1))
            if not os.path.exists(iterdir):
                os.makedirs(iterdir)
            saver.save(sess,'{}/Model.ckpt'.format(iterdir))
        print()

Epoch 1 starting..
Epoch 1 / Batch (1/17) / Accuracy 0.6612 / Loss 1.565960
Epoch 1 / Batch (2/17) / Accuracy 0.6356 / Loss 1.563752
Epoch 1 / Batch (3/17) / Accuracy 0.6306 / Loss 1.557487
Epoch 1 / Batch (4/17) / Accuracy 0.6662 / Loss 1.543100
Epoch 1 / Batch (5/17) / Accuracy 0.6837 / Loss 1.535537
Epoch 1 / Batch (6/17) / Accuracy 0.6575 / Loss 1.532863
Epoch 1 / Batch (7/17) / Accuracy 0.6538 / Loss 1.526457
Epoch 1 / Batch (8/17) / Accuracy 0.6938 / Loss 1.510095
Epoch 1 / Batch (9/17) / Accuracy 0.6869 / Loss 1.500139
Epoch 1 / Batch (10/17) / Accuracy 0.6900 / Loss 1.499742
Epoch 1 / Batch (11/17) / Accuracy 0.6619 / Loss 1.504568
Epoch 1 / Batch (12/17) / Accuracy 0.6625 / Loss 1.490370
Epoch 1 / Batch (13/17) / Accuracy 0.6700 / Loss 1.477415
Epoch 1 / Batch (14/17) / Accuracy 0.6913 / Loss 1.458761
Epoch 1 / Batch (15/17) / Accuracy 0.6525 / Loss 1.473176
Epoch 1 / Batch (16/17) / Accuracy 0.6869 / Loss 1.440466
Epoch 1 / Batch (17/17) / Accuracy 0.6675 / Loss 1.442846
Epoc

Epoch 9 / Batch (2/17) / Accuracy 0.7312 / Loss 1.292797
Epoch 9 / Batch (3/17) / Accuracy 0.7462 / Loss 1.270664
Epoch 9 / Batch (4/17) / Accuracy 0.7781 / Loss 1.258448
Epoch 9 / Batch (5/17) / Accuracy 0.7350 / Loss 1.292024
Epoch 9 / Batch (6/17) / Accuracy 0.7306 / Loss 1.282674
Epoch 9 / Batch (7/17) / Accuracy 0.7063 / Loss 1.307787
Epoch 9 / Batch (8/17) / Accuracy 0.7344 / Loss 1.288567
Epoch 9 / Batch (9/17) / Accuracy 0.7375 / Loss 1.272294
Epoch 9 / Batch (10/17) / Accuracy 0.7456 / Loss 1.266671
Epoch 9 / Batch (11/17) / Accuracy 0.7544 / Loss 1.266473
Epoch 9 / Batch (12/17) / Accuracy 0.7419 / Loss 1.281301
Epoch 9 / Batch (13/17) / Accuracy 0.7500 / Loss 1.271522
Epoch 9 / Batch (14/17) / Accuracy 0.7619 / Loss 1.252979
Epoch 9 / Batch (15/17) / Accuracy 0.7506 / Loss 1.264576
Epoch 9 / Batch (16/17) / Accuracy 0.7369 / Loss 1.297869
Epoch 9 / Batch (17/17) / Accuracy 0.7531 / Loss 1.276466
Epoch 9 complete, 1.0407726764678955 seconds elapsed

Epoch 10 starting..
Epoch 

Epoch 16 / Batch (17/17) / Accuracy 0.7694 / Loss 1.241420
Epoch 16 complete, 0.9665806293487549 seconds elapsed

Epoch 17 starting..
Epoch 17 / Batch (1/17) / Accuracy 0.7887 / Loss 1.201246
Epoch 17 / Batch (2/17) / Accuracy 0.7437 / Loss 1.237322
Epoch 17 / Batch (3/17) / Accuracy 0.7644 / Loss 1.214215
Epoch 17 / Batch (4/17) / Accuracy 0.7350 / Loss 1.244028
Epoch 17 / Batch (5/17) / Accuracy 0.7800 / Loss 1.234909
Epoch 17 / Batch (6/17) / Accuracy 0.7606 / Loss 1.205139
Epoch 17 / Batch (7/17) / Accuracy 0.7794 / Loss 1.209166
Epoch 17 / Batch (8/17) / Accuracy 0.7569 / Loss 1.225098
Epoch 17 / Batch (9/17) / Accuracy 0.7519 / Loss 1.237085
Epoch 17 / Batch (10/17) / Accuracy 0.7600 / Loss 1.245333
Epoch 17 / Batch (11/17) / Accuracy 0.7538 / Loss 1.230974
Epoch 17 / Batch (12/17) / Accuracy 0.7831 / Loss 1.212474
Epoch 17 / Batch (13/17) / Accuracy 0.7763 / Loss 1.223621
Epoch 17 / Batch (14/17) / Accuracy 0.7831 / Loss 1.188378
Epoch 17 / Batch (15/17) / Accuracy 0.7381 / Loss

Epoch 24 / Batch (12/17) / Accuracy 0.7912 / Loss 1.195240
Epoch 24 / Batch (13/17) / Accuracy 0.7700 / Loss 1.219836
Epoch 24 / Batch (14/17) / Accuracy 0.7875 / Loss 1.174740
Epoch 24 / Batch (15/17) / Accuracy 0.7825 / Loss 1.209765
Epoch 24 / Batch (16/17) / Accuracy 0.7594 / Loss 1.217687
Epoch 24 / Batch (17/17) / Accuracy 0.7581 / Loss 1.195721
Epoch 24 complete, 0.9598815441131592 seconds elapsed

Epoch 25 starting..
Epoch 25 / Batch (1/17) / Accuracy 0.7700 / Loss 1.223264
Epoch 25 / Batch (2/17) / Accuracy 0.7850 / Loss 1.205808
Epoch 25 / Batch (3/17) / Accuracy 0.7506 / Loss 1.201639
Epoch 25 / Batch (4/17) / Accuracy 0.7894 / Loss 1.198968
Epoch 25 / Batch (5/17) / Accuracy 0.7613 / Loss 1.206881
Epoch 25 / Batch (6/17) / Accuracy 0.7794 / Loss 1.205650
Epoch 25 / Batch (7/17) / Accuracy 0.7638 / Loss 1.187518
Epoch 25 / Batch (8/17) / Accuracy 0.7856 / Loss 1.180415
Epoch 25 / Batch (9/17) / Accuracy 0.7619 / Loss 1.214332
Epoch 25 / Batch (10/17) / Accuracy 0.7731 / Loss

Epoch 32 / Batch (6/17) / Accuracy 0.7638 / Loss 1.206735
Epoch 32 / Batch (7/17) / Accuracy 0.7631 / Loss 1.213583
Epoch 32 / Batch (8/17) / Accuracy 0.7581 / Loss 1.211651
Epoch 32 / Batch (9/17) / Accuracy 0.7944 / Loss 1.210428
Epoch 32 / Batch (10/17) / Accuracy 0.7812 / Loss 1.169764
Epoch 32 / Batch (11/17) / Accuracy 0.7763 / Loss 1.198035
Epoch 32 / Batch (12/17) / Accuracy 0.7725 / Loss 1.218791
Epoch 32 / Batch (13/17) / Accuracy 0.7694 / Loss 1.197435
Epoch 32 / Batch (14/17) / Accuracy 0.7919 / Loss 1.192048
Epoch 32 / Batch (15/17) / Accuracy 0.7681 / Loss 1.186935
Epoch 32 / Batch (16/17) / Accuracy 0.7975 / Loss 1.179054
Epoch 32 / Batch (17/17) / Accuracy 0.7937 / Loss 1.165834
Epoch 32 complete, 0.9603354930877686 seconds elapsed

Epoch 33 starting..
Epoch 33 / Batch (1/17) / Accuracy 0.7531 / Loss 1.222862
Epoch 33 / Batch (2/17) / Accuracy 0.7950 / Loss 1.198550
Epoch 33 / Batch (3/17) / Accuracy 0.7650 / Loss 1.184816
Epoch 33 / Batch (4/17) / Accuracy 0.7887 / Los

...

Epoch 370 / Batch (5/17) / Accuracy 0.8637 / Loss 1.101217
Epoch 370 / Batch (6/17) / Accuracy 0.8587 / Loss 1.120101
Epoch 370 / Batch (7/17) / Accuracy 0.8694 / Loss 1.076379
Epoch 370 / Batch (8/17) / Accuracy 0.8044 / Loss 1.159100
Epoch 370 / Batch (9/17) / Accuracy 0.8644 / Loss 1.097742
Epoch 370 / Batch (10/17) / Accuracy 0.8525 / Loss 1.122799
Epoch 370 / Batch (11/17) / Accuracy 0.8419 / Loss 1.141752
Epoch 370 / Batch (12/17) / Accuracy 0.8481 / Loss 1.129527
Epoch 370 / Batch (13/17) / Accuracy 0.8425 / Loss 1.146014
Epoch 370 / Batch (14/17) / Accuracy 0.8363 / Loss 1.139953
Epoch 370 / Batch (15/17) / Accuracy 0.8575 / Loss 1.103370
Epoch 370 / Batch (16/17) / Accuracy 0.8106 / Loss 1.155912
Epoch 370 / Batch (17/17) / Accuracy 0.8562 / Loss 1.096022
Epoch 370 complete, 0.9709148406982422 seconds elapsed

Epoch 371 starting..
Epoch 371 / Batch (1/17) / Accuracy 0.8444 / Loss 1.124445
Epoch 371 / Batch (2/17) / Accuracy 0.8506 / Loss 1.095966
Epoch 371 / Batch (3/17) / Acc

Epoch 377 / Batch (16/17) / Accuracy 0.8419 / Loss 1.106320
Epoch 377 / Batch (17/17) / Accuracy 0.8288 / Loss 1.141032
Epoch 377 complete, 0.9665586948394775 seconds elapsed

Epoch 378 starting..
Epoch 378 / Batch (1/17) / Accuracy 0.8719 / Loss 1.091245
Epoch 378 / Batch (2/17) / Accuracy 0.8462 / Loss 1.103700
Epoch 378 / Batch (3/17) / Accuracy 0.8388 / Loss 1.134632
Epoch 378 / Batch (4/17) / Accuracy 0.8469 / Loss 1.138022
Epoch 378 / Batch (5/17) / Accuracy 0.8363 / Loss 1.136633
Epoch 378 / Batch (6/17) / Accuracy 0.8512 / Loss 1.147587
Epoch 378 / Batch (7/17) / Accuracy 0.8644 / Loss 1.113261
Epoch 378 / Batch (8/17) / Accuracy 0.8294 / Loss 1.121465
Epoch 378 / Batch (9/17) / Accuracy 0.8650 / Loss 1.096331
Epoch 378 / Batch (10/17) / Accuracy 0.8519 / Loss 1.092509
Epoch 378 / Batch (11/17) / Accuracy 0.8631 / Loss 1.086613
Epoch 378 / Batch (12/17) / Accuracy 0.8469 / Loss 1.120150
Epoch 378 / Batch (13/17) / Accuracy 0.8450 / Loss 1.103423
Epoch 378 / Batch (14/17) / Accu

Epoch 385 / Batch (9/17) / Accuracy 0.8756 / Loss 1.065460
Epoch 385 / Batch (10/17) / Accuracy 0.8681 / Loss 1.110800
Epoch 385 / Batch (11/17) / Accuracy 0.8413 / Loss 1.114933
Epoch 385 / Batch (12/17) / Accuracy 0.8681 / Loss 1.105500
Epoch 385 / Batch (13/17) / Accuracy 0.8381 / Loss 1.115681
Epoch 385 / Batch (14/17) / Accuracy 0.8481 / Loss 1.114992
Epoch 385 / Batch (15/17) / Accuracy 0.8494 / Loss 1.096521
Epoch 385 / Batch (16/17) / Accuracy 0.8687 / Loss 1.111953
Epoch 385 / Batch (17/17) / Accuracy 0.8750 / Loss 1.089527
Epoch 385 complete, 1.1230502128601074 seconds elapsed

Epoch 386 starting..
Epoch 386 / Batch (1/17) / Accuracy 0.8331 / Loss 1.129081
Epoch 386 / Batch (2/17) / Accuracy 0.8838 / Loss 1.068902
Epoch 386 / Batch (3/17) / Accuracy 0.8506 / Loss 1.117550
Epoch 386 / Batch (4/17) / Accuracy 0.8375 / Loss 1.097516
Epoch 386 / Batch (5/17) / Accuracy 0.8506 / Loss 1.136447
Epoch 386 / Batch (6/17) / Accuracy 0.8475 / Loss 1.107992
Epoch 386 / Batch (7/17) / Acc

Epoch 393 / Batch (3/17) / Accuracy 0.8338 / Loss 1.156356
Epoch 393 / Batch (4/17) / Accuracy 0.8394 / Loss 1.135022
Epoch 393 / Batch (5/17) / Accuracy 0.8531 / Loss 1.099263
Epoch 393 / Batch (6/17) / Accuracy 0.8844 / Loss 1.067097
Epoch 393 / Batch (7/17) / Accuracy 0.8594 / Loss 1.089413
Epoch 393 / Batch (8/17) / Accuracy 0.8669 / Loss 1.090999
Epoch 393 / Batch (9/17) / Accuracy 0.8481 / Loss 1.103993
Epoch 393 / Batch (10/17) / Accuracy 0.8500 / Loss 1.134875
Epoch 393 / Batch (11/17) / Accuracy 0.8656 / Loss 1.077594
Epoch 393 / Batch (12/17) / Accuracy 0.8487 / Loss 1.070183
Epoch 393 / Batch (13/17) / Accuracy 0.8631 / Loss 1.087232
Epoch 393 / Batch (14/17) / Accuracy 0.8625 / Loss 1.096554
Epoch 393 / Batch (15/17) / Accuracy 0.8569 / Loss 1.128054
Epoch 393 / Batch (16/17) / Accuracy 0.8537 / Loss 1.103607
Epoch 393 / Batch (17/17) / Accuracy 0.8575 / Loss 1.114526
Epoch 393 complete, 1.1401538848876953 seconds elapsed

Epoch 394 starting..
Epoch 394 / Batch (1/17) / Acc

Epoch 400 / Batch (14/17) / Accuracy 0.8606 / Loss 1.098709
Epoch 400 / Batch (15/17) / Accuracy 0.8456 / Loss 1.111030
Epoch 400 / Batch (16/17) / Accuracy 0.8331 / Loss 1.138228
Epoch 400 / Batch (17/17) / Accuracy 0.8531 / Loss 1.109047
Epoch 400 complete, 1.0213844776153564 seconds elapsed

